# 🌐 Chapter 3: Word Embeddings

## 📌 Overview  
(Recall) Bag of Words (BoW) and TF-IDF represent words as sparse vectors where each dimension corresponds to a unique word. However, they fail to capture **semantic relationships** between words (e.g., "king" and "queen").

**Word Embeddings** Word embeddings are a way to represent words as numerical vectors, where words with similar meanings or contexts are represented by vectors that are close together in space. This allows computers to understand and analyze the semantic relationships between words in a way that's more intuitive than simply using one-hot encoding or other basic methods. 

 To represent the complexity of a typical 50,000 word English vocabulary requires hundreds of features. Designing all those features by hand, and assigning accurate coordinates to all those words, would be a lot of work! Therefore, A typical embedding might use a 300 dimensional space, so each word would be represented by 300 numbers.

- Reference : https://www.cs.cmu.edu/~dst/WordEmbeddingDemo/tutorial.html

**Word Embeddings** solve this problem by mapping words into dense, low-dimensional vectors where similar words have similar representations. These embeddings are learned from large text corpora.

Popular embedding methods:
- Word2Vec (Skip-gram, CBOW)
    - **CBOW**: Predict center word from context words.
    - **Skip-Gram**: Predict context words from center word.
- GloVe (Global Vectors)
- FastText



## 2. Key Components
| Component      | Meaning                                              |
|----------------|------------------------------------------------------|
| One-hot vector | Input where only one element is 1, others are 0       |
| W1 (input)     | (V x d) matrix: one-hot → hidden dense vector         |
| W2 (output)    | (d x V) matrix: hidden vector → output vocabulary     |

## 3. Mathematical Flow
Given:
- Vocabulary size = **V**
- Embedding size = **d**

**Step-by-Step Computation:**
1. Input: One-hot vector x (size V)
2. Hidden Layer: h = W1ᵀ x
3. Output Layer: u = W2ᵀ h
4. Softmax: p = softmax(u)
5. Loss: Cross-entropy
6. Update W1 and W2


## 🚀 Word2Vec (Skip-gram with Negative Sampling)

---

### 🟢 Vocabulary and Indexing

Given corpus: `"The cat sits"`  
Vocabulary size $V = 3$.

| Word   | Index |
|--------|-------|
| the    | 0     |
| cat    | 1     |
| sits   | 2     |

---

### 🟡 Step 1: Input Embedding Matrix $W$ (Center Words)

- Shape: $V \times d = 3 \times 2$
- Each **row** represents the embedding vector for a word **when it is the center word**.

$$
W =
\begin{bmatrix}
0.2 & -0.1 \\
0.7 & 0.3 \\
-0.5 & 0.6
\end{bmatrix}
$$

- $u_{\text{the}} = [0.2, -0.1]$
- $u_{\text{cat}} = [0.7, 0.3]$
- $u_{\text{sits}} = [-0.5, 0.6]$

---

#### **Mathematical Concept:**
The input embedding matrix $W$ maps one-hot encoded center words to dense embeddings.  
For a one-hot vector $x_{w_t}$ for center word $w_t$:

$$
u_{w_t} = W^\top x_{w_t}
$$

This operation selects the $w_t$-th row of $W$ directly.

---

### 🟠 Step 2: Output Embedding Matrix $W'$ (Context Words, different from the input weights W)

- Shape: $V \times d = 3 \times 2$
- Each **row** represents the embedding vector for a word **when it is a context word**.

$$
W' =
\begin{bmatrix}
0.1 & 0.4 \\
-0.3 & 0.5 \\
0.6 & -0.2
\end{bmatrix}
$$

- $v_{\text{the}} = [0.1, 0.4]$
- $v_{\text{cat}} = [-0.3, 0.5]$
- $v_{\text{sits}} = [0.6, -0.2]$

---

#### **Mathematical Concept:**
The output embedding matrix $W'$ holds context word vectors.  
During prediction, the center word's embedding (from $W$) is compared with these context vectors (from $W'$) via dot product.

---

### 🟣 Step 3: Dot Product Calculation (Score Computation)

Center word = `"cat"` (index 1):

$$
u_{\text{cat}} = [0.7, 0.3]
$$

The score between center word $u_{\text{cat}}$ and each context word $v_{w_o}$ is:

$$
s_{w_o} = u_{\text{cat}}^\top v_{w_o}
$$

---

#### **Detailed Calculations:**

1. Between `"cat"` (center) and `"the"` (context):

$$
s_{\text{the}} = (0.7 \times 0.1) + (0.3 \times 0.4) = 0.07 + 0.12 = 0.19
$$

2. Between `"cat"` (center) and `"cat"` (context):

$$
s_{\text{cat}} = (0.7 \times -0.3) + (0.3 \times 0.5) = -0.21 + 0.15 = -0.06
$$

3. Between `"cat"` (center) and `"sits"` (context):

$$
s_{\text{sits}} = (0.7 \times 0.6) + (0.3 \times -0.2) = 0.42 - 0.06 = 0.36
$$

---

### 🟢 Step 4: Resulting Raw Scores Table

| Center Word | Context Word | Dot Product Score |
|-------------|--------------|------------------|
| cat         | the          | 0.19             |
| cat         | cat          | -0.06            |
| cat         | sits         | 0.36             |

---

### ⚡ Mathematical Concept Behind Dot Product Scoring

The dot product measures **alignment** or **similarity** between the embedding vectors of the center and context words:

$$
s_{w_o} = \sum_{i=1}^{d} u_{w_t}^{(i)} \cdot v_{w_o}^{(i)}
$$

- Large positive dot product → vectors point in a similar direction (high similarity).
- Zero → vectors are orthogonal (unrelated).
- Negative → vectors point in opposite directions (dissimilar).

These scores are:
- Passed into the **softmax function** to obtain probabilities:

$$
p(w_o \mid w_t) = \frac{\exp(s_{w_o})}{\sum_{w=1}^{V} \exp(s_w)}
$$

- Or used directly for **negative sampling loss** in more efficient training.

#### Apply Softmax to Get Probabilities

Softmax formula:

$$
p(w_o \mid w_t) = \frac{\exp(s_{w_o})}{\sum_{w=1}^{V} \exp(s_{w})}
$$

Exponentials:
- $\exp(0.19) \approx 1.209$
- $\exp(-0.06) \approx 0.942$
- $\exp(0.36) \approx 1.433$

Denominator:

$$
1.209 + 0.942 + 1.433 = 3.584
$$

Probabilities:
- $p(\text{the} \mid \text{cat}) \approx 0.337$
- $p(\text{cat} \mid \text{cat}) \approx 0.263$
- $p(\text{sits} \mid \text{cat}) \approx 0.400$

---

### 🟢 Step 5: Calculate Loss (True Context is "the")

Loss:

$$
L = -\log p(\text{the} \mid \text{cat})
$$

Substituting:

$$
L = -\log(0.337) \approx 1.087
$$

---

### 🟠 Step 6: Negative Sampling Alternative (Optional)

If using negative sampling:
- Positive pair: ("cat", "the")
- Negative pairs: ("cat", "cat") and ("cat", "sits")

Loss:

$$
L = -\log \sigma(u_{\text{cat}}^\top v_{\text{the}}) - \sum_{i=1}^{k} \log \sigma(-u_{\text{cat}}^\top v_{\text{neg}, i})
$$

Where:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

---

### ✅ Conclusion

- The model maximizes dot product scores for correct pairs.
- Reduces scores for incorrect pairs.
- Learns meaningful word embeddings where similar words have similar vector representations.

🧠 Why This Works:

- The model **encourages higher scores** for true (center, context) pairs.
- It **reduces scores** for randomly sampled negative (incorrect) pairs.
- Over time, embeddings for words appearing together in similar contexts **move closer in the vector space**.

---


# 📚 Full Step-by-Step with Full Mathematical Analysis + Examples
## 1. Setup

Vocabulary = `["apple", "banana", "cat", "dog"]`

| Word     | Index |
|:---------|:------|
| apple    | 0     |
| banana   | 1     |
| cat      | 2     |
| dog      | 3     |

Vocabulary size **V = 4**
Embedding size **d = 2**

Initial weight matrices:

**W1** (V × d):

| Word   | dim1  | dim2  |
|--------|-------|-------|
| apple  | 0.5   | -0.2  |
| banana | 0.3   | 0.8   |
| cat    | -0.4  | -0.4  |
| dog    | 1.2   | 0.5   |

**W2** (d × V):

| dim    | apple | banana | cat  | dog  |
|--------|-------|--------|------|------|
| dim1   | -0.3  | 0.6    | -0.5 | -0.4 |
| dim2   | 0.2   | -0.9   | -1.0 | -0.2 |
## 2. Iteration 1: Center Word = `"cat"`, Context Word = `"dog"`

### Step 1: Create One-Hot Vector

```
x = [0, 0, 1, 0]
```

### Step 2: Hidden Layer Calculation

```
h = W1ᵀ x = W1["cat"] = [-0.4, -0.4]
```

### Step 3: Output Layer Calculation

```
u = W2ᵀ h
```

Softmax logits:

```
u = [0.04, 0.12, 0.6, 0.24]
```

### Step 4: Softmax Calculation

```
p = [0.198, 0.214, 0.346, 0.242]
```

### Step 5: Loss Calculation

Cross-Entropy Loss:

```
Loss = -log(p_dog) ≈ 1.418
```

### Step 6: Backpropagation

Error Vector:

```
error = p - y = [0.198, 0.214, 0.346, -0.758]
```

Gradients:

- `grad_W2 = h outer error`
- `grad_W1 = W2 @ error`

### Step 7: Update W1 and W2

Gradient descent:

```
W1["cat"] = W1["cat"] - learning_rate * grad_W1
W2 = W2 - learning_rate * grad_W2
```
## 3. Full Mathematical Explanation of Cross-Entropy and Softmax

### 1. Softmax Function

```
softmax(uᵢ) = exp(uᵢ) / Σ exp(uⱼ)
```

### 2. Cross-Entropy Loss

```
L = -sum_i (yᵢ * log(pᵢ))
```

Simplified when y is one-hot:

```
L = -log(pₖ)
```

### 3. Derivative of Cross-Entropy + Softmax

```
∂L/∂uᵢ = pᵢ - yᵢ
```

Thus:

```
error = p - y
```

### 4. Gradients

- Gradient w.r.t W2:
  
```
∂L/∂W2 = h outer (p - y)
```

- Gradient w.r.t W1:
  
```
∂L/∂h = W2 (p - y)
```

thus only update W1[center_word].

### 5. Update Rules

```
W2 = W2 - learning_rate * grad_W2
W1[center_word] = W1[center_word] - learning_rate * grad_W1
```

In [8]:
import numpy as np

# Set initial W1 and W2
W1 = np.array([
    [0.5, -0.2],
    [0.3, 0.8],
    [-0.4, -0.4],
    [1.2, 0.5]
])

W2 = np.array([
    [-0.3, 0.6, -0.5, -0.4],
    [0.2, -0.9, -1.0, -0.2]
])

vocab = ["apple", "banana", "cat", "dog"]
word_to_index = {word: idx for idx, word in enumerate(vocab)}

# Center word = "cat", Context word = "dog"
center_word = "cat"
context_word = "dog"

# One-hot vector
V = 4
x = np.zeros(V)
x[word_to_index[center_word]] = 1

# Hidden layer
h = W1.T @ x
print("Hidden vector h:", h)

# Output logits
u = W2.T @ h
print("Logits u:", u)

# Softmax
def softmax(z):
    e_z = np.exp(z - np.max(z))
    return e_z / e_z.sum()

p = softmax(u)
print("Predicted probabilities p:", p)

# Cross-Entropy Loss
true_context_index = word_to_index[context_word]
loss = -np.log(p[true_context_index])
print("Loss:", loss)

# Error
error = p.copy()
error[true_context_index] -= 1
print("Error vector:", error)

# Gradients
grad_W2 = np.outer(h, error)
grad_W1 = W2 @ error
print("Gradient W2:\n", grad_W2)
print("Gradient W1:", grad_W1)

# Update
learning_rate = 0.1
W2 -= learning_rate * grad_W2
W1[word_to_index[center_word]] -= learning_rate * grad_W1

print("\nUpdated W1:\n", W1)
print("Updated W2:\n", W2)

Hidden vector h: [-0.4 -0.4]
Logits u: [0.04 0.12 0.6  0.24]
Predicted probabilities p: [0.19780976 0.21428475 0.3463001  0.24160538]
Loss: 1.420449526685425
Error vector: [ 0.19780976  0.21428475  0.3463001  -0.75839462]
Gradient W2:
 [[-0.0791239  -0.0857139  -0.13852004  0.30335785]
 [-0.0791239  -0.0857139  -0.13852004  0.30335785]]
Gradient W1: [ 0.19943572 -0.34791551]

Updated W1:
 [[ 0.5        -0.2       ]
 [ 0.3         0.8       ]
 [-0.41994357 -0.36520845]
 [ 1.2         0.5       ]]
Updated W2:
 [[-0.29208761  0.60857139 -0.486148   -0.43033578]
 [ 0.20791239 -0.89142861 -0.986148   -0.23033578]]



## Python Code  
**Idea:** Words that appear in similar contexts have similar embeddings (distributional hypothesis).  
- **CBOW (Continuous Bag of Words):** Predicts a word from surrounding context.  
- **Skip-gram:** Predicts surrounding context from a given word.

**Example using `gensim`:**
Gensim is a Python library for topic modeling and document similarity analysis. It provides efficient implementations of algorithms like Latent Semantic Analysis (LSA), Latent Dirichlet Allocation (LDA), and word2vec for discovering semantic structures in large text corpora.

The role of Gensim in text analysis are as follows:

- Text preprocessing: Gensim provides functions for preprocessing text data, including tokenization, normalization, stemming, and lemmatization, ensuring that the text is cleaned and standardized for further analysis.

- Document Representation: Gensim allows users to represent documents as vectors in a high-dimensional space, facilitating various text analysis tasks such as document clustering, classification, and similarity analysis.

- Word Embeddings: Gensim includes implementations of the word2vec, GloVe algorithm, which learns distributed representations of words in a vector space, capturing semantic relationships and similarities between words, facilitating tasks such as semantic similarity calculation, word analogy reasoning, and language understanding.

- Topic Modeling: Gensim includes implementations of algorithms such as Latent Semantic Analysis (LSA), Latent Dirichlet Allocation (LDA) or Non-negative Matrix Factorization (NMF) for topic modeling, enabling users to discover underlying topics within large text corpora.
Document Similarity and Retrieval: Gensim provides functionality for computing similarities between documents based on their content, facilitating tasks such as document clustering, similarity analysis, and information retrieval.

Overall, Gensim is a powerful library for discovering semantic structures in text data, offering efficient implementations of Text preprocessing,Document Representation, Word Embeddings, topic modeling, document similarity and Retrieval:. Its scalability and ease of use make it a popular choice for researchers and practitioners working with large text corpora.

In [2]:
%pip install gensim

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 41.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 38.9 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
from gensim.models import Word2Vec  # Import Word2Vec
from nltk.tokenize import word_tokenize  # Tokenizer for splitting sentences

# Sample corpus
sentences = [
    "Natural language processing is fun",
    "Machine learning is a part of artificial intelligence",
    "Word embeddings capture semantic meaning"
]

# Tokenize the corpus (split each sentence into words)
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

# Train the Word2Vec model using Skip-gram (sg=1) with vector size of 50
model = Word2Vec(sentences=tokenized_sentences, vector_size=50, window=2, min_count=1, sg=1)

# Get the embedding vector for the word 'machine'
print("Embedding for 'machine':\n", model.wv['machine'])

# Check similarity between two words
print("Similarity between 'machine' and 'learning':", model.wv.similarity('machine', 'learning'))

/Users/moka/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Embedding for 'machine':
 [-0.01648536  0.01859871 -0.00039532 -0.00393455  0.00920726 -0.00819063
  0.00548623  0.01387993  0.01213085 -0.01502159  0.0187647   0.00934362
  0.00793224 -0.01248701  0.01691996 -0.00430033  0.01765038 -0.01072401
 -0.01625884  0.01364912  0.00334239 -0.00439702  0.0190272   0.01898771
 -0.01954809  0.00501046  0.01231338  0.00774491  0.00404557  0.000861
  0.00134726 -0.00764127 -0.0142805  -0.00417774  0.0078478   0.01763737
  0.0185183  -0.01195187 -0.01880534  0.01952875  0.00685957  0.01033223
  0.01256469 -0.00560853  0.01464541  0.00566054  0.00574201 -0.00476074
 -0.0062565  -0.00474028]
Similarity between 'machine' and 'learning': 0.11255005


# 2️⃣ GloVe Embeddings

Idea: Uses global word co-occurrence statistics across the entire corpus instead of local context windows.

Example: Using Pretrained GloVe Embeddings (Common Crawl or Wikipedia):

In [6]:
# Download pretrained GloVe from: https://nlp.stanford.edu/projects/glove/
# Example: 'glove.6B.50d.txt' contains 50-dimensional vectors
# the word 'to' in txt file has the value as below : 
# to = [0.68047 -0.039263 0.30186 -0.17792 0.42962 0.032246 -0.41376 0.13228 .... -0.064699 -0.26044] with the length of 50

import numpy as np

# Load the GloVe embeddings (assuming the file 'glove.6B.50d.txt' is downloaded)
glove_embeddings = {}
with open('data/glove.6B.50d.txt', 'r', encoding='utf-8') as f: # text file saved in 'data' folder
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = vector

# Example: Get embedding for 'machine'
print("GloVe embedding for 'machine':\n", glove_embeddings.get('machine'))


GloVe embedding for 'machine':
 [-0.34165   -0.81267    1.4513     0.05914   -0.080801   0.39567
  0.10064   -0.5468    -0.18887    0.11364   -0.040956  -0.5637
 -0.32191    0.15968   -0.59756   -0.14571   -0.77074    1.2955
 -0.72002   -0.90818    0.76644    0.05346   -0.0031632 -0.15341
  0.22065   -1.191     -1.0775    -0.29768    1.327     -0.51359
  2.6229    -0.67411   -0.82558    0.14283   -0.014214   0.90775
  0.66828    0.48431    0.1543     0.26044    1.0191     0.015872
 -0.75325    0.58992    0.4546    -0.19678    0.42138   -0.43168
  0.11985    0.14094  ]


# 3️⃣ FastText Embeddings

Idea: Considers subword information (character n-grams), helping handle out-of-vocabulary (OOV) words better than Word2Vec and GloVe.

FastText can generate embeddings for unseen words based on their subword components.

Example:

In [5]:
from gensim.models import FastText  # Import FastText

# Use the same tokenized corpus
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

# Train FastText model
fasttext_model = FastText(sentences=tokenized_sentences, vector_size=50, window=2, min_count=1)

# Get vector for a word
print("FastText embedding for 'machine':\n", fasttext_model.wv['machine'])


FastText embedding for 'machine':
 [-1.8079143e-03  2.1979981e-03 -1.7691230e-03  5.6677201e-04
  1.2654356e-03  2.1495651e-03 -1.5551907e-03  4.3800026e-03
 -2.0496619e-03  5.3299527e-04 -2.5257603e-03  1.6439921e-03
  3.4959912e-03 -9.7273325e-05  2.1534071e-03  2.0341277e-03
 -5.6405651e-04  1.8103337e-03  4.1687866e-03  7.4633321e-04
 -3.0441333e-03 -3.0280279e-03  3.9849104e-03 -7.3370530e-04
 -1.7331528e-03  1.6396311e-03 -6.8702095e-04 -2.2539324e-03
  5.6145241e-04  1.4721482e-03 -2.8888162e-03 -2.2243629e-03
  2.1639713e-03 -1.2766268e-03  6.0394765e-03  4.9851830e-03
  3.3022531e-03  1.5956949e-03 -4.3668048e-03  1.5206628e-03
 -2.3396676e-03  7.1912521e-04  2.4290388e-03 -5.5817286e-03
  2.9966333e-03 -6.4665275e-03 -5.4450257e-04 -2.2184665e-03
  1.3568229e-03 -5.2718865e-03]


## 🧩 Why Use Word Embeddings?

| Method          | Sparse/Dense    | Captures Word Meaning? | Handles OOV Words?      |
|-----------------|-----------------|-----------------------|------------------------|
| BoW / TF-IDF    | Sparse           | ❌ No                  | ❌ No                   |
| Word2Vec        | Dense            | ✅ Yes                 | ❌ No                   |
| GloVe           | Dense            | ✅ Yes                 | ❌ No                   |
| FastText        | Dense            | ✅ Yes                 | ✅ Yes (via subwords)   |


## ✅ Answers to Practice Questions (Word Embeddings)

### 1️⃣ Why are word embeddings better than BoW or TF-IDF for capturing meaning?
Word embeddings (like Word2Vec, GloVe, FastText) map words into dense vectors where **semantically similar words are closer together in the vector space**. Unlike BoW or TF-IDF, which only count word occurrences and ignore word order or meaning, embeddings capture relationships between words (e.g., "king" is close to "queen", "Paris" is close to "France").

---

### 2️⃣ What is the main difference between Skip-gram and CBOW in Word2Vec?
- **CBOW (Continuous Bag of Words):** Predicts the target word based on its surrounding context words.
- **Skip-gram:** Predicts the surrounding context words given the target word.
- Typically, **Skip-gram works better for small datasets** and rare words, while **CBOW is faster on large datasets**.

---

### 3️⃣ How does FastText handle words that it has not seen during training?
FastText breaks words into **subword units (character n-grams)**. This allows it to create word vectors by combining the vectors of these subwords. Even if a word was not in the training data (out-of-vocabulary, OOV), FastText can generate a vector based on its subword pieces, making it more robust to rare or unseen words.

Example:  
The word **"running"** may be broken into subwords like `"run"`, `"unn"`, `"nni"`, `"nin"`, `"ing"`.

---


## 📚 Python Full Step-by-Step Breakdown

In [7]:
# 1. What is Word2Vec?

# Word2Vec learns a vector (embedding) for each word so that similar words are close together in vector space.
# Two main models:
# - CBOW: Predict center word from context words
# - Skip-Gram: Predict context words from center word

# 2. Key Components

# | Component      | Meaning                                              |
# |----------------|------------------------------------------------------|
# | One-hot vector | Input where only one element is 1, others are 0       |
# | W1 (input)     | (V x d) matrix: one-hot → hidden dense vector         |
# | W2 (output)    | (d x V) matrix: hidden vector → output vocabulary     |

# 3. Mathematical Flow

# Given:
# - Vocabulary size = V
# - Embedding size = d

# Step-by-Step Computation:
# 1. Input: One-hot vector x (size V)
# 2. Hidden Layer: h = W1^T x
# 3. Output Layer: u = W2^T h
# 4. Softmax: p = softmax(u)
# 5. Loss: Cross-entropy
# 6. Update W1 and W2

# 4. Example Setup

# Vocabulary = ["apple", "banana", "cat", "dog"]
# V = 4, d = 2

import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Vocabulary
vocab = ["apple", "banana", "cat", "dog"]
word_to_index = {word: idx for idx, word in enumerate(vocab)}
index_to_word = {idx: word for word, idx in word_to_index.items()}

V = len(vocab)
d = 2

# Initial weight matrices
W1 = np.random.randn(V, d)
W2 = np.random.randn(d, V)

print("Initial W1:\n", W1)
print("Initial W2:\n", W2)

# 5. Iteration 1: Center word "cat", Context word "dog"

center_word = "cat"
context_word = "dog"

# One-hot vector for "cat"
x = np.zeros(V)
x[word_to_index[center_word]] = 1

# Step 1: Hidden Layer
h = W1.T @ x  # or simply W1[cat]
print("\nHidden vector h (cat):", h)

# Step 2: Output Layer
u = W2.T @ h
print("Output logits u:", u)

# Step 3: Softmax

def softmax(z):
    e_z = np.exp(z - np.max(z))  # for numerical stability
    return e_z / e_z.sum()

p = softmax(u)
print("Predicted probabilities p:", p)

# Step 4: Compute Loss
true_context_index = word_to_index[context_word]
loss = -np.log(p[true_context_index])
print("Loss:", loss)

# Step 5: Backpropagation (Simple Gradient Update)
learning_rate = 0.1

# Gradients
error = p.copy()
error[true_context_index] -= 1  # p - y (one-hot)

# Gradients w.r.t W2 and W1
grad_W2 = np.outer(h, error)
grad_W1 = W2 @ error

# Update W1 and W2
W1[word_to_index[center_word]] -= learning_rate * grad_W1
W2 -= learning_rate * grad_W2

print("\nUpdated W1:\n", W1)
print("Updated W2:\n", W2)

# 6. Iteration 2: Center word "dog", Context word "barks"

# For this example, assume "barks" is treated as a known word (for simplicity).
# Here, let's use "apple" as the pretend context word.
center_word = "dog"
context_word = "apple"

# One-hot vector for "dog"
x = np.zeros(V)
x[word_to_index[center_word]] = 1

# Step 1: Hidden Layer
h = W1.T @ x
print("\nHidden vector h (dog):", h)

# Step 2: Output Layer
u = W2.T @ h
print("Output logits u:", u)

# Step 3: Softmax
p = softmax(u)
print("Predicted probabilities p:", p)

# Step 4: Compute Loss
true_context_index = word_to_index[context_word]
loss = -np.log(p[true_context_index])
print("Loss:", loss)

# Step 5: Backpropagation
error = p.copy()
error[true_context_index] -= 1

grad_W2 = np.outer(h, error)
grad_W1 = W2 @ error

# Update W1 and W2
W1[word_to_index[center_word]] -= learning_rate * grad_W1
W2 -= learning_rate * grad_W2

print("\nUpdated W1 after 2nd iteration:\n", W1)
print("Updated W2 after 2nd iteration:\n", W2)

### 🔥 Summary

# - W1 maps one-hot vectors to dense hidden vectors (word embeddings).
# - W2 maps hidden vectors back to vocabulary.
# - Both W1 and W2 are updated every iteration to reduce prediction error.
# - After many iterations, W1 contains meaningful word embeddings!


Initial W1:
 [[ 0.49671415 -0.1382643 ]
 [ 0.64768854  1.52302986]
 [-0.23415337 -0.23413696]
 [ 1.57921282  0.76743473]]
Initial W2:
 [[-0.46947439  0.54256004 -0.46341769 -0.46572975]
 [ 0.24196227 -1.91328024 -1.72491783 -0.56228753]]

Hidden vector h (cat): [-0.23415337 -0.23413696]
Output logits u: [0.0532767  0.32092735 0.51237783 0.24070448]
Predicted probabilities p: [0.19624449 0.25647006 0.31058649 0.23669895]
Loss: 1.4409661987967817

Updated W1:
 [[ 0.49671415 -0.1382643 ]
 [ 0.64768854  1.52302986]
 [-0.26001131 -0.17916127]
 [ 1.57921282  0.76743473]]
Updated W2:
 [[-0.46487925  0.54856538 -0.45614521 -0.48360271]
 [ 0.24655708 -1.90727533 -1.71764585 -0.58015923]]

Hidden vector h (dog): [1.57921282 0.76743473]
Output logits u: [-0.54492681 -0.59740786 -2.03853144 -1.20894593]
Predicted probabilities p: [0.37199444 0.35297522 0.08353572 0.19149463]
Loss: 0.9888763844670431

Updated W1 after 2nd iteration:
 [[ 0.49671415 -0.1382643 ]
 [ 0.64768854  1.52302986]
 [-0.260011